# بسم الله الرحمن الرحیم

# محمد مهدی شفیقی - پروژه نهایی درس مباحث ویژه

# تحلیل شبکه همکاری علمی CA-HepTh

این پروژه به تحلیل ویژگی‌های ساختاری و دینامیکی شبکه هم‌نویسندگی در حوزه فیزیک انرژی بالا می‌پردازد.

In [ ]:
# فقط در صورت نیاز اجرا کن
!pip install --quiet networkx pandas numpy matplotlib python-louvain tqdm scipy
# برای عملکرد بهتر (اختیاری، سریع‌تر و مقیاس‌پذیرتر)
!pip install --quiet python-igraph leidenalg


## تنظیم مسیر داده — مسیر را طبق محل واقعی تغییر بده


In [2]:
DATA_FOLDER = "../data/cit-HepTh-abstracts"   # محتوای پوشه: 1992/, 1993/, ...


# ایمپورت‌های پایه


In [3]:
import os, re, gc, pickle, math, time
from itertools import combinations
from collections import defaultdict
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# توابع مفید


In [4]:
def save_pickle(obj, path):
    with open(path, "wb") as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_pickle(path):
    with open(path, "rb") as f:
        return pickle.load(f)


# چک محیط و نمونه‌برداری فایل‌ها + تابع استخراج نویسنده (اجرای اولیه)

هدف این بلوک این است که بفهمیم ساختار فایل‌های .abs چگونه است، تعداد فایل‌ها در هر پوشه چقدر است، و یک تابع استخراجِ نویسندهٔ مقاوم/قوی بنویسیم که روی دادهٔ واقعی کار کند — بعد از اجرای این بلوک به‌راحتی می‌توانیم بقیهٔ پردازش (ساخت گراف زمانی) را ایمن اجرا کنیم.

In [6]:
# A-0: Environment check + sample parsing of .abs files
# اجرا در یک سلول جدید در Jupyter notebook

import os, re, time
from collections import defaultdict
from itertools import combinations
try:
    from tqdm.auto import tqdm
except Exception:
    # tqdm اختیاری است؛ اگر نصب نیست، از آن صرفنظر می‌کنیم
    def tqdm(x, **_): 
        return x

# مسیرِ پوشه‌ی cit-HepTh-abstracts را بر اساس ساختار تو تنظیم کن:
DATA_ROOT = "../data/cit-HepTh-abstracts"   # اگر مسیرت متفاوت است این را تغییر بده

# 1) شمارش فایل‌ها و نام پوشه‌ها (سال‌ها)
years = sorted([d for d in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, d))])
print("Found year-folders (sample):", years[:6], " ... total:", len(years))

year_file_counts = {}
total_files = 0
for y in years:
    p = os.path.join(DATA_ROOT, y)
    files = [f for f in os.listdir(p) if f.endswith('.abs')]
    year_file_counts[y] = len(files)
    total_files += len(files)

print(f"Total .abs files: {total_files}")
print("Files per year (first 10):")
for y in years[:10]:
    print(f"  {y}: {year_file_counts[y]} files")

# 2) Robust author-extraction function (tries چند الگو)
_author_patterns = [
    re.compile(r'Authors:\s*(.*?)\n(?:Title:|Comments:|\\\n|$)', re.DOTALL | re.IGNORECASE),
    re.compile(r'Authors:\s*(.*)', re.IGNORECASE),
]

def extract_authors_from_text(txt):
    """
    تلاش می کند رشته‌ی نویسندگان را از متن استخراج کند.
    بازگشتی: لیست اسامی نویسنده‌ها (هر اسم تمیزشده)
    """
    # بعضی فایل‌ها از backslash \\ برای جدا کردن بخش‌ها استفاده می‌کنند، آن‌ها را به newline تبدیل کن
    t = txt.replace('\\\n', '\n').replace('\\', '\n')
    authors_text = None
    for pat in _author_patterns:
        m = pat.search(t)
        if m:
            authors_text = m.group(1)
            break
    if not authors_text:
        # fallback: خطی جستجو کن
        for line in t.splitlines():
            if line.strip().lower().startswith("authors:"):
                authors_text = line.split(':',1)[1]
                break
    if not authors_text:
        return []  # هیچ نویسنده‌ای پیدا نشد

    # پاک‌سازی و جداسازی: "and" و کاما و ; را مدنظر قرار بده
    authors_text = authors_text.replace('\n', ' ')
    # بعضی فرمت‌ها "A and B" دارند
    authors_text = re.sub(r'\sand\s', ',', authors_text)
    # جداکننده‌ها: ',' یا ';' یا ' and '
    parts = re.split(r',|;|\band\b', authors_text)
    authors = []
    for p in parts:
        p = p.strip()
        if not p:
            continue
        # حذف موارد غیرِ اسم (مثل affiliation داخل پرانتز)
        p = re.sub(r'\(.*?\)', '', p).strip()
        # normalize spaces
        p = re.sub(r'\s+', ' ', p)
        authors.append(p)
    # unique preserving order
    seen = set()
    authors_clean = []
    for a in authors:
        key = a.lower()
        if key not in seen:
            seen.add(key)
            authors_clean.append(a)
    return authors_clean

# 3) نمایش چند نمونه فایل و نتیجه‌ی استخراج نویسنده
SAMPLES_TO_SHOW = 6
sample_files = []
for y in years:
    d = os.path.join(DATA_ROOT, y)
    files = [f for f in os.listdir(d) if f.endswith('.abs')]
    if files:
        sample_files.append((y, files[:1][0], os.path.join(d, files[0])))
    if len(sample_files) >= SAMPLES_TO_SHOW:
        break

print("\nSample files and extracted authors:")
for y, fname, fpath in sample_files:
    try:
        with open(fpath, 'r', encoding='utf-8', errors='ignore') as fh:
            txt = fh.read(4000)  # فقط 4k اول برای نمایش
    except Exception as e:
        txt = f"(error reading: {e})"
    authors = extract_authors_from_text(txt)
    print(f"\nYear {y} | file: {fname}")
    print(" Extracted authors:", authors[:10])

# 4) (اختیاری) تست زمان خواندن تعداد نمونه‌ای از فایل‌ها
NTEST = 200   # تعداد فایل‌ها که برای سنجش سرعت پردازش بررسی می‌کنیم
t0 = time.time()
count = 0
for y in years:
    d = os.path.join(DATA_ROOT, y)
    for fn in os.listdir(d):
        if not fn.endswith('.abs'):
            continue
        count += 1
        if count > NTEST:
            break
    if count > NTEST:
        break
t1 = time.time()
print(f"\nRough IO check: enumerated ~{min(count, NTEST)} files in {t1-t0:.2f}s")

print("\nA-0 done. If sample author extraction looks reasonable, reply with 'A-0 OK' and we'll run A-1: build efficient temporal-edge lists (ids + per-year CSV dump).")


Found year-folders (sample): ['1992', '1993', '1994', '1995', '1996', '1997']  ... total: 12
Total .abs files: 29555
Files per year (first 10):
  1992: 1367 files
  1993: 2058 files
  1994: 2377 files
  1995: 2303 files
  1996: 2606 files
  1997: 2673 files
  1998: 2758 files
  1999: 2803 files
  2000: 3126 files
  2001: 3153 files

Sample files and extracted authors:

Year 1992 | file: 9201001.abs
 Extracted authors: ['C. Itzykson', 'J.-B. Zuber']

Year 1993 | file: 9301001.abs
 Extracted authors: ['G.K.Savvidy', 'K.G.Savvidy']

Year 1994 | file: 9401001.abs
 Extracted authors: ['Jorge Ananias Neto']

Year 1995 | file: 9501001.abs
 Extracted authors: []

Year 1996 | file: 9601001.abs
 Extracted authors: []

Year 1997 | file: 9701001.abs
 Extracted authors: ['M. Zyskin We consider d-dimensional Riemanian manifolds which admit d-2 commuting space-like Killing vector fields', 'orthogonal to a surface', 'containing two one-parametric families of light-like curves. The condition of the Ric

# ساخت IDها و فایل‌های یال (per-year CSV) — حافظه‌پسند

هدف این گام:

به هر نویسنده یک شناسه‌ی عددی (int ID) نسبت دهیم (map: author -> id) تا پردازش‌های بعدی سریع و حافظه‌پسند شوند.

برای هر سال یک فایل CSV بسازیم که هر سطرش یک یالِ بدون‌جهت (undirected) را به‌صورت src_id,dst_id,year ذخیره کند.

یال‌ها در هر سال بدون تکرار (deduped) باشند و جفت‌ها به‌صورت مرتب شده (min_id,max_id) ذخیره شوند تا تکرار‌های (u,v) و (v,u) یکسان شناخته شوند.

یک فایل nodes.csv بسازیم که id → author_name نگه دارد (برای مرجع و مصورسازی بعدی).

In [7]:
# A-1: Build temporal edge lists (author -> id map + per-year deduped CSVs)
import os, re, time, csv
from itertools import combinations
from collections import defaultdict, OrderedDict

# مسیر ورودی (پوشه cit-HepTh-abstracts) — اگر مسیرت فرق داره اینجا را تغییر بده
DATA_ROOT = "../data/cit-HepTh-abstracts"
OUT_DIR = "../data/edges_by_year"   # خروجی: فایل‌های CSV برای هر سال و nodes.csv

os.makedirs(OUT_DIR, exist_ok=True)

# تابع استخراج نویسنده را از A-0 بیاور (یا از src/data_parser.py)
# اگر در نوت بوک قبلی تعریف شده می‌توان مستقیماً استفاده کرد؛ در غیر این صورت این پیاده‌سازی را کپی کن:
def extract_authors_from_text(txt):
    t = txt.replace('\\\n', '\n').replace('\\', '\n')
    # الگوهای ساده
    m = re.search(r'Authors:\s*(.*?)\n(?:Title:|Comments:|\\\n|$)', t, re.DOTALL | re.IGNORECASE)
    authors_text = None
    if m:
        authors_text = m.group(1)
    else:
        # fallback خطی
        for line in t.splitlines():
            if line.strip().lower().startswith("authors:"):
                authors_text = line.split(':',1)[1]
                break
    if not authors_text:
        return []
    authors_text = authors_text.replace('\n', ' ')
    authors_text = re.sub(r'\sand\s', ',', authors_text)
    parts = re.split(r',|;|\band\b', authors_text)
    authors = []
    for p in parts:
        p = p.strip()
        if not p:
            continue
        p = re.sub(r'\(.*?\)', '', p).strip()
        p = re.sub(r'\s+', ' ', p)
        authors.append(p)
    seen = set(); out=[]
    for a in authors:
        k = a.lower()
        if k not in seen:
            seen.add(k); out.append(a)
    return out

# 1) scan years (sorted)
years = sorted([d for d in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, d))])
print("Years found:", years)

# global author -> id mapping (OrderedDict to keep insertion order stable)
author2id = OrderedDict()
next_id = 0

# We'll store for each year a set of edges (tuple (u_id, v_id) with u<v)
# For memory-safety: we'll use a set per year (dataset small enough). If memory problem داشتیم،
# می‌توانستیم flush به فایل موقت و merge پس از آن انجام دهیم.
year_edge_counts = {}
start = time.time()

for y in years:
    year_dir = os.path.join(DATA_ROOT, y)
    edge_set = set()
    files = [f for f in os.listdir(year_dir) if f.endswith('.abs')]
    # progress print
    t0 = time.time()
    for i,fn in enumerate(files):
        fpath = os.path.join(year_dir, fn)
        with open(fpath, 'r', encoding='utf-8', errors='ignore') as fh:
            txt = fh.read()
        authors = extract_authors_from_text(txt)
        if len(authors) < 2:
            continue
        # assign ids
        ids = []
        for a in authors:
            key = a.strip()
            if key not in author2id:
                author2id[key] = next_id
                ids.append(next_id)
                next_id += 1
            else:
                ids.append(author2id[key])
        # all pairs (undirected clique)
        for u,v in combinations(ids, 2):
            if u == v:
                continue
            a,b = (u,v) if u < v else (v,u)
            edge_set.add((a,b))
    # write deduped per-year csv
    out_csv = os.path.join(OUT_DIR, f"edges_{y}.csv")
    with open(out_csv, 'w', newline='', encoding='utf-8') as csvf:
        writer = csv.writer(csvf)
        # header optional
        writer.writerow(["src","dst","year"])
        for (u,v) in sorted(edge_set):
            writer.writerow([u,v,y])
    year_edge_counts[y] = len(edge_set)
    print(f"Year {y}: processed {len(files)} files -> {year_edge_counts[y]} unique edges  (time: {time.time()-t0:.2f}s)")

# write nodes file
nodes_file = os.path.join(OUT_DIR, "nodes.csv")
with open(nodes_file, 'w', newline='', encoding='utf-8') as nf:
    w = csv.writer(nf)
    w.writerow(["id","author"])
    for author,aid in author2id.items():
        w.writerow([aid, author])

total_time = time.time() - start
print("\nDone. Total authors (unique):", len(author2id))
print("Per-year edge counts:", year_edge_counts)
print("Nodes file saved to:", nodes_file)
print("Per-year CSVs saved to:", OUT_DIR)
print(f"Elapsed time: {total_time:.2f}s")


Years found: ['1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003']
Year 1992: processed 1367 files -> 1911 unique edges  (time: 2.02s)
Year 1993: processed 2058 files -> 2673 unique edges  (time: 2.19s)
Year 1994: processed 2377 files -> 3131 unique edges  (time: 1.44s)
Year 1995: processed 2303 files -> 3588 unique edges  (time: 1.83s)
Year 1996: processed 2606 files -> 4876 unique edges  (time: 1.53s)
Year 1997: processed 2673 files -> 3823 unique edges  (time: 1.51s)
Year 1998: processed 2758 files -> 4340 unique edges  (time: 1.64s)
Year 1999: processed 2803 files -> 4595 unique edges  (time: 1.52s)
Year 2000: processed 3126 files -> 4755 unique edges  (time: 1.45s)
Year 2001: processed 3153 files -> 5064 unique edges  (time: 1.77s)
Year 2002: processed 3312 files -> 6656 unique edges  (time: 1.70s)
Year 2003: processed 1019 files -> 2240 unique edges  (time: 0.50s)

Done. Total authors (unique): 16715
Per-year edge counts: {'1992': 1911, 

الان ما یک نسخه تمیز و بهینه از یال‌های زمانی داریم که:

۱۶٬۷۱۵ نویسنده یکتا (id → name)

یال‌های منحصر به فرد برای هر سال بدون تکرار

همه‌چیز آماده برای ساخت snapshot‌ های گراف و تحلیل شاخص‌هاست

حالا می‌توانیم وارد A-2 شویم:

ساخت گراف برای هر سال از روی CSVها

محاسبه شاخص‌های پایه برای هر snapshot (Average Degree, Density, Clustering Coefficient، اندازه مولفه بزرگ، Approx Diameter، Approx Betweenness، PageRank و غیره)

ذخیره نتایج در یک جدول (DataFrame) برای استفاده در تحلیل رشد و ترسیم نمودارها